# Projet du cours "Kernel methods" du master MVA
Le projet de ce cours consiste en la classification d'images. Dans ce notebook, on implémente de manière la plus détaillée et la plus claire possible l'ensemble des classes et fonctions des algorithmes que nous utilisons.    

Ce projet se fera en 3 étapes :
- Preprocessing des données
- Feature extraction
- Prediction par SVM

Il est indiqué dans l'énoncé que l'étape de preprocessing a déjà été réalisée. Nous allons donc directement procéder à l'extraction de nos features (par HOG) et à la prédiction de la nature de nos images (par SVM).

Ce notebook utilise comme kernel le RBF kernel qui est celui qui donne les meilleurs résultats. Pour voir les implémentations réalisées avec d'autres kernels, on pourra se reporter aux notebooks joints (moins détaillés et n'effectuant pas toutes les étapes ci-dessous mais présentant notamment les implémentations liées à chaque kernel considéré).

## Chargement des modules

In [1]:
%pylab inline
import pandas
import cvxopt
import time
import numpy as np
import numpy.linalg as la
import matplotlib.pyplot as plt 
import scipy
from scipy.stats import mode
from scipy.linalg import eigh

Populating the interactive namespace from numpy and matplotlib


# Importation des données

On importe les données du concours. On peut aussi insérer à la place de Xtr la base obtenue en appliquant les ondelettes pour voir les résultats obtenus avec les ondelettes.

In [3]:
df_train=pandas.read_csv('/Users/badr-eddinecherief-abdellatif/Documents/ENSAE/3A/MVA/Kernel methods/Projet ENS/Bases/Xtr.csv',header=None,sep=',')
df_train=df_train.drop(3072,1)
df_test=pandas.read_csv('/Users/badr-eddinecherief-abdellatif/Documents/ENSAE/3A/MVA/Kernel methods/Projet ENS/Bases/Xte.csv',header=None,sep=',')
df_test=df_test.drop(3072,1)
y_train=pandas.read_csv('/Users/badr-eddinecherief-abdellatif/Documents/ENSAE/3A/MVA/Kernel methods/Projet ENS/Bases/Ytr.csv',sep=';')
y_train=pandas.DataFrame(y_train['Prediction'])

In [4]:
X_train = np.genfromtxt('/Users/badr-eddinecherief-abdellatif/Documents/ENSAE/3A/MVA/Kernel methods/Projet ENS/Bases/Xtr.csv',delimiter=',')[:,0:3072]
Y_train = np.genfromtxt('/Users/badr-eddinecherief-abdellatif/Documents/ENSAE/3A/MVA/Kernel methods/Projet ENS/Bases/Ytr.csv',delimiter=';')[1:5001,1]
X_test = np.genfromtxt('/Users/badr-eddinecherief-abdellatif/Documents/ENSAE/3A/MVA/Kernel methods/Projet ENS/Bases/Xte.csv',delimiter=',')[:,0:3072]

# Répartition des labels
Nous travaillons sur un échantillon d'apprentissage dans le but de classifier des images. Vérifions que nous avons bien équirépartition des images dans l'échantillon d'apprentissage afin de pouvoir mener à bien notre modélisation

In [5]:
y_train["Prediction"].value_counts()

7    500
3    500
6    500
2    500
9    500
5    500
1    500
8    500
4    500
0    500
Name: Prediction, dtype: int64

# Implémentation des méthodes

Dans cette partie, nous implémenterons les différentes méthodes que nous testerons par la suite. Il s'agit des méthodes suivantes:
- SVM
- ACP
- HOG



## Support Vector Machine

In [6]:
class SVMTrainer(object):
    def __init__(self, c, sigma):
        self._c = c
        self.sigma=sigma
        
    def train(self, X, y):
        lagrange_multipliers = self._compute_multipliers(X, y)
        return self._construct_predictor(X, y, lagrange_multipliers)

    def _gram_matrix(self, X):
        n_samples, n_features = X.shape
        v=norm(X,2,axis=1)**2
        v2=array([v,]*n_samples)+array([v,]*n_samples).transpose()-2*dot(X.values,transpose(X.values))
        K=exp(-v2/(2*(self.sigma)**2)) 
        print("Taille de la matrice de Gram :",K.shape)
        return K

    def _construct_predictor(self, X, y, lagrange_multipliers):
        support_vector_indices=abs(lagrange_multipliers)>0
        support_multipliers = lagrange_multipliers[support_vector_indices] # alpha i non nuls
        support_vectors = X[support_vector_indices] # vecteurs supports
        support_vector_labels = y[support_vector_indices] # labels des vecteurs supports
        print("Nombre de vecteurs supports :",len(support_vectors))
        pre=SVMPredictor(sigma=self.sigma,bias=0.0,weights=support_multipliers,
             support_vectors=support_vectors,support_vector_labels=support_vector_labels).predictest(support_vectors.values)[0]
        bias = np.mean(support_vector_labels[:,0]-pre)
        print("Biais:",bias)
        return SVMPredictor(sigma=self.sigma,bias=bias,weights=support_multipliers,support_vectors=support_vectors,
            support_vector_labels=support_vector_labels)

    def _compute_multipliers(self, X, y):
        n_samples, n_features = X.shape
        K=self._gram_matrix(X)
        P=cvxopt.matrix(K)
        q=cvxopt.matrix(-1*y[:,0])
        G_std=cvxopt.matrix(-1*np.diag(y[:,0]))
        h_std=cvxopt.matrix(np.zeros(n_samples))
        G_slack=cvxopt.matrix(np.diag(y[:,0]))
        h_slack=cvxopt.matrix(np.ones(n_samples) * self._c)
        G=cvxopt.matrix(np.vstack((G_std, G_slack)))
        h=cvxopt.matrix(np.vstack((h_std, h_slack)))
        A=cvxopt.matrix(np.ones(n_samples),(1,n_samples))
        b=cvxopt.matrix(0.0)
        solution = cvxopt.solvers.qp(P, q, G, h, A=None, b=None)
        return np.ravel(solution['x'])

In [7]:
class SVMPredictor(object):
    def __init__(self,sigma,bias,weights,support_vectors,support_vector_labels):
        self.sigma=sigma
        self._bias = bias
        self._weights = weights
        self._support_vectors = support_vectors
        self._support_vector_labels = support_vector_labels

    def predictest(self,x):
        v = array([norm(x,2,axis=1)**2,]*len(self._support_vectors))
        inter = array([norm(self._support_vectors,2,axis=1)**2,]*len(x)).transpose() + v
        inter1 = inter - 2*dot(self._support_vectors,transpose(x))
        inter2 = exp(-asarray(inter1)/(2*(self.sigma)**2))
        result = dot(transpose(self._weights),inter2)+self._bias
        return sign(result),result

## Adaptation du SVM au cas multi-classe
On va tout d'abord implémenter la méthode One Vs One, i.e. considérer 10x9/2=45 problèmes de classifications binaires en traitant des paires d'instances, puis implémenter la méthode One Vs Rest, i.e. entraîner un classifieur par classe.

### One Vs One

In [8]:
def SVMOvO(data_train,data_test,is_train,C,sigma):
    SVM=SVMTrainer(C,sigma)
    result=(20*ones(len(data_test))).reshape(len(data_test),1)
    for i in [0,1,2,3,4,5,6,7,8]:
        for j in range(i+1,10):
            print("Classe :",i,j)
            df_train_ij=data_train[asmatrix(is_train,dtype=ndarray)==(i,j)].copy()
            is_train_ij=is_train[asmatrix(is_train,dtype=ndarray)==(i,j)].copy()
            is_train_ij[is_train==i]=1
            is_train_ij[is_train==j]=-1
            s1=time.clock()
            SVMPred=SVM.train(df_train_ij,asfarray(is_train_ij))
            s2=time.clock()
            print("Temps pour l'entrainement :",s2-s1)
            s3=time.clock()
            values=SVMPred.predictest(data_test.values)[0]
            s4=time.clock()
            print("Temps pour la prédiction :",s4-s3)
            values[values==1]=i
            values[values==-1]=j
            values=values.reshape(len(data_test),1)
            result=concatenate((result,values),1)
            is_test_predict=mode(result,axis=1)
    return(is_test_predict)    

### One Vs Rest

In [9]:
def SVMOvR(data_train,data_test,is_train,C,sigma):
    SVM=SVMTrainer(C,sigma)
    result=(-20*ones(len(data_test))).reshape(len(data_test),1)
    for i in [0,1,2,3,4,5,6,7,8,9]:
        print("Classe :",i)
        is_train_i=is_train.copy()
        is_train_i[is_train==i]=1
        is_train_i[is_train!=i]=-1
        s1=time.clock()
        SVMPred=SVM.train(data_train,asfarray(is_train_i))
        s2=time.clock()
        print("Temps pour l'entrainement :",s2-s1)
        s3=time.clock()
        values=SVMPred.predictest(data_test.values)[1]
        s4=time.clock()
        print("Temps pour la prédiction :",s4-s3)
        values=values.reshape(len(data_test),1)
        result=concatenate((result,values),1)
    predict=argmax(result,axis=1)-1
    return(predict)  

# Kernel - Analyse en Composantes Principales

Simple Kernel-ACP suivant les étapes du slide 163 du cours.

In [10]:
def kpca(X, sigma, n_components):
    
    # Gram matrix 
    n_samples, n_features = X.shape
    v=norm(X,2,axis=1)**2
    v2=array([v,]*n_samples)+array([v,]*n_samples).transpose()-2*dot(X.values,transpose(X.values))
    K=exp(-v2/(2*sigma**2))
    
    # Center 
    N = K.shape[0]
    one_n = np.ones((N,N)) / N
    K = K - one_n.dot(K) - K.dot(one_n) + one_n.dot(K).dot(one_n)

    # Eigenvalues in descending order with corresponding eigenvectors 
    eigvals, eigvecs = eigh(K)

    # Normalize
    vect = eigvecs//np.sqrt(eigvals)
    vect = np.column_stack((vect[:,-i] for i in range(1,n_components+1)))
    
    X_kpca = dot(K,vect)
    return(real(X_kpca))

# HOG

Implémentation de la méthode d'extraction de features inspirée de l'article https://hal.inria.fr/inria-00548512/document de N. Dalal et B. Triggs et du site http://www.learnopencv.com/histogram-of-oriented-gradients/. On choisit de diviser l'image de 32 pixels en 4 cellules de (8,8) pixels chacun, en normalisant par blocs de 4 cellules, ce qui donne une base X de taille 3 (nombre de blocs sur l'axe x) fois 3 (nombre de blocs sur l'axe y) fois 9 (taille d'un histogramme) fois 4 (nombre d'histogrammes par bloc, un histogramme correspondant à une cellule) fois 3 (pour les différentes couleurs de l'image), i.e. on obtient une base X avec 972 variables au lieu de 3072.

In [11]:
class HOG :

    def __init__(self, nbins=9):
        self.nbins = nbins

    def _calc_gradient_discret(self, img):
        n_x, n_y = img.shape
        histogram = numpy.zeros((4, 4, self.nbins))

        for i in range(0, n_x) :
            for j in range(0, n_y) :
                dx = 0
                dy = 0
                if i < n_x - 1 :
                    dx += img[i + 1, j]
                if i > 0 :
                    dx -= img[i - 1, j]
                if j < n_y - 1 :
                    dy += img[i, j + 1]
                if j > 0 :
                    dy -= img[i, j - 1]

                if dy == 0 and dx == 0 :
                    continue

                magnitude = numpy.sqrt(dx**2 + dy**2)
                if dx == 0 :
                    angle = numpy.pi / 2
                else:
                    angle = numpy.arctan(dy / dx)
                    angle = (angle + numpy.pi / 2) / (numpy.pi / self.nbins)
               
                bin_pos = int(numpy.floor(angle))

                if bin_pos == self.nbins :
                    bin_pos = 0
                    angle = 0

                closest_bin = bin_pos

                if bin_pos == 0 :
                    if angle < 0.5 :
                        second_closest_bin = self.nbins - 1
                    else:
                        second_closest_bin = 1
                elif bin_pos == self.nbins - 1 :
                    if angle < self.nbins - 0.5 :
                        second_closest_bin = self.nbins - 2
                    else:
                        second_closest_bin = 0
                else:
                    if angle < bin_pos + 0.5 :
                        second_closest_bin = bin_pos - 1
                    else:
                        second_closest_bin = bin_pos + 1

                if angle < bin_pos + 0.5 :
                    second_closest_bin_distance = angle - (bin_pos - 0.5)
                else:
                    second_closest_bin_distance = (bin_pos + 1.5) - angle

                r = second_closest_bin_distance
                histogram[i / 8, j / 8, closest_bin] += r * magnitude
                histogram[i / 8, j / 8, second_closest_bin] += (1 - r) * magnitude

        vec = numpy.zeros((3, 3, self.nbins * 4))

        for i in range(3):
            for j in range(3):
                aux = histogram[i:i + 2, j:j + 2, :].flatten().copy()
                aux = aux / numpy.linalg.norm(aux)
                vec[i, j, :] = aux

        return vec.flatten()

    
    def _calc_gradient_one_image(self, img):
        nchannels = img.shape[2]
        vec = []

        for i in range(nchannels):
            vec.append(self._calc_gradient_discret(img[:,:,i]))

        return numpy.array(vec).flatten()

    
    def hog(self, X):
        n = X.shape[0]
        X_new = []

        for i in range(n):
            X_new.append(self._calc_gradient_one_image(X[i,:,:,:]))

        return numpy.array(X_new)

# Tests
Afin de tester nos modèles sur nos données d'entraînement, on va découper notre base d'entraînement en train/test à hauteur de 60/40%.

Pour plus de clarté, on ne présente ici que les modèles optimaux pour chacune des étapes suivantes (SVM One-Vs-Rest avant HOG, après ACP, après HOG, après HOG+ACP), i.e. la méthode optimale avec les paramètres optimaux parmi un nombre fini de modèles (les 2 paramètres variant de 1 à 10). Pour s'en assurer, il suffit de faire varier les paramètres à chaque fois (par une simple boucle sur ces paramètres) pour chacune des méthodes. La partie finale "Submit" utilise la méthode optimale (après HOG mais sans ACP).

### Avant HOG : SVM One Vs Rest (C=1,sigma=2) 

In [12]:
X_train_1 = df_train[0:3000]
X_test_1 = df_train[3000:5000]
Y_train_1 = y_train[0:3000]
Y_test_1 = y_train[3000:5000]
print(X_train_1.shape)
print(X_test_1.shape)
Y_train_1["Prediction"].value_counts()

(3000, 3072)
(2000, 3072)


3    311
1    308
0    306
6    305
8    304
4    298
5    294
7    292
9    291
2    291
Name: Prediction, dtype: int64

In [13]:
y_predict_OvR=SVMOvR(X_train_1,X_test_1,Y_train_1,1,2)

Classe : 0
Taille de la matrice de Gram : (3000, 3000)
     pcost       dcost       gap    pres   dres
 0: -7.5775e+02 -6.9036e+03  3e+04  3e+00  6e-15
 1: -5.5753e+02 -4.1051e+03  5e+03  1e-01  6e-15
 2: -5.3667e+02 -9.0202e+02  4e+02  6e-03  5e-15
 3: -5.5776e+02 -6.5443e+02  1e+02  6e-04  5e-15
 4: -5.6998e+02 -5.9357e+02  2e+01  2e-05  6e-15
 5: -5.7408e+02 -5.7772e+02  4e+00  2e-06  6e-15
 6: -5.7498e+02 -5.7532e+02  3e-01  1e-07  6e-15
 7: -5.7509e+02 -5.7511e+02  2e-02  5e-09  6e-15
 8: -5.7509e+02 -5.7509e+02  6e-04  1e-10  7e-15
 9: -5.7509e+02 -5.7509e+02  3e-05  2e-12  7e-15
Optimal solution found.
Nombre de vecteurs supports : 3000
Biais: 0.203333333333
Temps pour l'entrainement : 54.454631000000006
Temps pour la prédiction : 2.1874090000000024
Classe : 1
Taille de la matrice de Gram : (3000, 3000)
     pcost       dcost       gap    pres   dres
 0: -5.8153e+02 -6.1508e+03  3e+04  2e+00  7e-15
 1: -4.5759e+02 -3.4482e+03  4e+03  1e-01  8e-15
 2: -4.4874e+02 -1.0120e+03  6e+

In [14]:
compt=Y_test_1.values[:,0]-y_predict_OvR
print("Pourcentage de bonnes prédictions :",sum(compt==0)/len(Y_test_1))

Pourcentage de bonnes prédictions : 0.2115


### Après ACP : SVM One Vs Rest (C=1,sigma=2, 30 composantes conservées) 

In [15]:
X_train_kacp = pandas.DataFrame(kpca(df_train,2,30))[0:3000]
X_test_kacp = pandas.DataFrame(kpca(df_train,2,30))[3000:5000]
Y_train_1 = y_train[0:3000]
Y_test_1 = y_train[3000:5000]
print(X_train_1.shape)
print(X_test_1.shape)
Y_train_1["Prediction"].value_counts()

(3000, 3072)
(2000, 3072)


3    311
1    308
0    306
6    305
8    304
4    298
5    294
7    292
9    291
2    291
Name: Prediction, dtype: int64

In [16]:
y_predict_OvR=SVMOvR(X_train_kacp,X_test_kacp,Y_train_1,1,2)

Classe : 0
Taille de la matrice de Gram : (3000, 3000)
     pcost       dcost       gap    pres   dres
 0: -1.3333e+03 -4.6667e+03  3e+03  0e+00  5e-16
 1: -1.4077e+03 -1.6223e+03  2e+02  1e-16  3e-16
 2: -1.4861e+03 -1.5169e+03  3e+01  2e-16  2e-16
 3: -1.4980e+03 -1.5022e+03  4e+00  2e-16  2e-16
 4: -1.4997e+03 -1.5003e+03  6e-01  2e-16  2e-16
 5: -1.5000e+03 -1.5000e+03  9e-02  2e-16  2e-16
 6: -1.5000e+03 -1.5000e+03  1e-02  2e-16  2e-16
 7: -1.5000e+03 -1.5000e+03  2e-03  2e-16  2e-16
 8: -1.5000e+03 -1.5000e+03  3e-04  2e-16  1e-16
Optimal solution found.
Nombre de vecteurs supports : 3000
Biais: 0.0
Temps pour l'entrainement : 36.493772000000035
Temps pour la prédiction : 0.4217270000000326
Classe : 1
Taille de la matrice de Gram : (3000, 3000)
     pcost       dcost       gap    pres   dres
 0: -1.3333e+03 -4.6667e+03  3e+03  0e+00  4e-16
 1: -1.4077e+03 -1.6223e+03  2e+02  1e-16  3e-16
 2: -1.4861e+03 -1.5169e+03  3e+01  2e-16  2e-16
 3: -1.4980e+03 -1.5022e+03  4e+00  2e-16  

In [17]:
compt=Y_test_1.values[:,0]-y_predict_OvR
print("Pourcentage de bonnes prédictions :",sum(compt==0)/len(Y_test_1))

Pourcentage de bonnes prédictions : 0.146


### Après HOG :  SVM One Vs Rest (C=1, sigma=2) 

In [19]:
# Création de la nouvelle base X obtenue après extraction de features

Xtrain = numpy.reshape(X_train, (X_train.shape[0], 3, 32, 32))
Xtrain = numpy.swapaxes(Xtrain, 1, 2)
Xtrain = numpy.swapaxes(Xtrain, 2, 3)

Xtest = numpy.reshape(X_test, (X_test.shape[0], 3, 32, 32))
Xtest = numpy.swapaxes(Xtest, 1, 2)
Xtest = numpy.swapaxes(Xtest, 2, 3)

histogram=HOG(nbins=9)
h=histogram.hog(Xtrain)
g=histogram.hog(Xtest)

X_tr = pandas.DataFrame(h)
X_te = pandas.DataFrame(g)

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:63: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:64: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [20]:
X_train_2 = X_tr[0:3000]
X_test_2 = X_tr[3000:5000]
Y_train_2 = y_train[0:3000]
Y_test_2 = y_train[3000:5000]
print(X_train_2.shape)
print(X_test_2.shape)
Y_train_2["Prediction"].value_counts()

(3000, 972)
(2000, 972)


3    311
1    308
0    306
6    305
8    304
4    298
5    294
7    292
9    291
2    291
Name: Prediction, dtype: int64

In [21]:
y_predict_OvR=SVMOvR(X_train_2,X_test_2,Y_train_2,1,2)

Classe : 0
Taille de la matrice de Gram : (3000, 3000)
     pcost       dcost       gap    pres   dres
 0: -4.1590e+02 -5.3848e+03  2e+04  2e+00  2e-15
 1: -3.4990e+02 -2.9285e+03  4e+03  2e-01  2e-15
 2: -3.3254e+02 -8.3656e+02  6e+02  2e-02  3e-15
 3: -3.6704e+02 -5.1380e+02  2e+02  5e-03  2e-15
 4: -3.8693e+02 -4.2974e+02  4e+01  1e-03  2e-15
 5: -3.9489e+02 -4.0666e+02  1e+01  5e-05  2e-15
 6: -3.9776e+02 -3.9985e+02  2e+00  4e-06  2e-15
 7: -3.9840e+02 -3.9862e+02  2e-01  3e-07  2e-15
 8: -3.9848e+02 -3.9849e+02  7e-03  5e-09  2e-15
 9: -3.9849e+02 -3.9849e+02  2e-04  9e-11  2e-15
Optimal solution found.
Nombre de vecteurs supports : 3000
Biais: 0.046
Temps pour l'entrainement : 48.13222700000006
Temps pour la prédiction : 0.9573359999999411
Classe : 1
Taille de la matrice de Gram : (3000, 3000)
     pcost       dcost       gap    pres   dres
 0: -4.0098e+02 -5.6251e+03  3e+04  2e+00  3e-15
 1: -3.3132e+02 -3.1783e+03  4e+03  2e-01  3e-15
 2: -3.1044e+02 -8.8245e+02  7e+02  3e-02 

In [22]:
compt=Y_test_2.values[:,0]-y_predict_OvR
print("Pourcentage de bonnes prédictions :",sum(compt==0)/len(Y_test_2))

Pourcentage de bonnes prédictions : 0.5625


### Après HOG + ACP :  SVM One Vs Rest (C=1, sigma=2, 30 composantes conservées) 

In [16]:
X_train_3 = pandas.DataFrame(kpca(X_tr,2,30))[0:3000]
X_test_3 = pandas.DataFrame(kpca(X_tr,2,30))[3000:5000]
Y_train_3 = y_train[0:3000]
Y_test_3 = y_train[3000:5000]
print(X_train_3.shape)
print(X_test_3.shape)
Y_train_3["Prediction"].value_counts()

(3000, 30)
(2000, 30)


3    311
1    308
0    306
6    305
8    304
4    298
5    294
7    292
9    291
2    291
Name: Prediction, dtype: int64

In [17]:
y_predict_OvR=SVMOvR(X_train_3,X_test_3,Y_train_3,1,2)

Classe : 0
Taille de la matrice de Gram : (3000, 3000)
     pcost       dcost       gap    pres   dres
 0:  1.1667e+03 -2.6167e+04  3e+04  0e+00  6e-16
 1: -1.0988e+03 -3.6326e+03  3e+03  2e-16  4e-16
 2: -1.4944e+03 -1.7071e+03  2e+02  1e-16  1e-16
 3: -1.5000e+03 -1.5029e+03  3e+00  2e-16  9e-17
 4: -1.5000e+03 -1.5000e+03  3e-02  1e-16  1e-16
 5: -1.5000e+03 -1.5000e+03  3e-04  1e-16  6e-18
Optimal solution found.
Nombre de vecteurs supports : 3000
Biais: 0.0
Temps pour l'entrainement : 29.016598000000158
Temps pour la prédiction : 0.4627500000001419
Classe : 1
Taille de la matrice de Gram : (3000, 3000)
     pcost       dcost       gap    pres   dres
 0:  1.1667e+03 -2.6167e+04  3e+04  0e+00  6e-16
 1: -1.0988e+03 -3.6326e+03  3e+03  2e-16  4e-16
 2: -1.4944e+03 -1.7071e+03  2e+02  1e-16  1e-16
 3: -1.5000e+03 -1.5029e+03  3e+00  2e-16  9e-17
 4: -1.5000e+03 -1.5000e+03  3e-02  1e-16  1e-16
 5: -1.5000e+03 -1.5000e+03  3e-04  1e-16  6e-18
Optimal solution found.
Nombre de vecteurs 

In [18]:
compt=Y_test_3.values[:,0]-y_predict_OvR
print("Pourcentage de bonnes prédictions :",sum(compt==0)/len(Y_test_3))

Pourcentage de bonnes prédictions : 0.3785


# On soumet nos résultats finaux sur le site Kaggle

### Le modèle optimal est un SVM One Vs Rest (paramètres C=1 et sigma=2) après HOG mais sans ACP

In [24]:
final_pred=SVMOvR(X_tr,X_te,y_train,1,2)

Classe : 0
Taille de la matrice de Gram : (5000, 5000)
     pcost       dcost       gap    pres   dres
 0: -7.5296e+02 -1.3907e+04  5e+04  1e+00  4e-15
 1: -6.6657e+02 -6.3442e+03  9e+03  2e-01  5e-15
 2: -6.4586e+02 -1.8752e+03  1e+03  2e-02  5e-15
 3: -7.0631e+02 -1.0787e+03  4e+02  4e-03  4e-15
 4: -7.4900e+02 -8.6273e+02  1e+02  3e-04  5e-15
 5: -7.6725e+02 -8.0158e+02  3e+01  4e-05  5e-15
 6: -7.7424e+02 -7.8190e+02  8e+00  3e-06  6e-15
 7: -7.7622e+02 -7.7735e+02  1e+00  1e-07  6e-15
 8: -7.7658e+02 -7.7664e+02  6e-02  4e-09  6e-15
 9: -7.7660e+02 -7.7660e+02  1e-03  6e-11  6e-15
10: -7.7660e+02 -7.7660e+02  3e-05  9e-13  6e-15
Optimal solution found.
Nombre de vecteurs supports : 5000
Biais: 0.0212
Temps pour l'entrainement : 205.1234430000004
Temps pour la prédiction : 1.1689519999999902
Classe : 1
Taille de la matrice de Gram : (5000, 5000)
     pcost       dcost       gap    pres   dres
 0: -6.7813e+02 -1.3758e+04  5e+04  1e+00  5e-15
 1: -5.6807e+02 -6.3784e+03  9e+03  2e-01

In [25]:
results = [int(i) for i in final_pred]
X_result = pandas.DataFrame(results,columns=['Prediction'])
X_result.index += 1
X_result.to_csv('Yte.csv',index=True,index_label='Id')